In [58]:
import pandas as pd


In [61]:
# Load the CSV file (replace the path with your own file location)
df = pd.read_excel('data/filtered_only(3).xlsx')
dfsanchuan = pd.read_excel('data/sanhuan.xlsx')
# Show the first 5 rows
must_have_data = ['Actuator size', 'Actuator system [mm]','Coil connection', 'Cv value [gal/min]', 'Electrical connection', 'Enclosure rating IP', 'Frequency [Hz]', 'Function',  'Inlet connection type', 'Inlet size [in]', 'Inlet size [mm]', 'Kv value [m³/h]', 'Outlet connection type', 'Outlet size [in]', 'Outlet size [mm]','Refrigerants', 'Supply voltage 50Hz Max [V]', 'Supply voltage 50Hz Min [V]', 'Supply voltage 60Hz Max [V]', 'Supply voltage 60Hz Min [V]', 'Supply voltage [V] AC', 'Supply voltage [V] AC [max]', 'Supply voltage [V] AC [min]', 'Supply voltage [V] DC']

df[must_have_data] = df[must_have_data].fillna('BLANK').replace('', 'BLANK').replace(' ', 'BLANK')



In [65]:
columns = df.columns.tolist()
print(columns)

# valve to target - search similar to this one
target_index = 0
target_index2 = 290
target_row = dfsanchuan.iloc[target_index]
print(target_row)
target_row2 = df.iloc[target_index2]
#df = df.drop(index=target_index)

# remove first 14 values
df = df.iloc[14:]
# print(len(df))

['<ID>', '<Object Type>', 'Product name', 'Headline', 'Actuator size', 'Actuator system [mm]', 'Ambient temperature', 'Ambient temperature range [°C] [max]', 'Ambient temperature range [°C] [min]', 'Approval', 'Cable length [m]', 'Coil', 'Coil connection', 'Coil type', 'Cv value [gal/min]', 'Cv value invert flow [gal/min]', 'Direction', 'Electrical connection', 'Enclosure rating IP', 'Frequency [Hz]', 'Function', 'Identical product', 'Inlet connection type', 'Inlet size [in]', 'Inlet size [mm]', 'Kv value [m³/h]', 'Kv value invert flow [m³/h]', 'Manual operation option', 'Max. Working Pressure [bar]', 'Max. Working Pressure [psig]', 'MaxOPD[bar] 10W AC', 'MaxOPD[bar] 12W AC', 'MaxOPD[bar] 14W AC', 'MaxOPD[bar] 20W DC', 'MaxOPD[bar] coil BJ/BX', 'MaxOPD[psi] 10W AC', 'MaxOPD[psi] 14W AC', 'MaxOPD[psi] 20W DC', 'MaxOPD[psi] coil BJ/BX', 'Medium temperature range [°C] [max]', 'Medium temperature range [°C] [min]', 'MinODP [bar]', 'MinODP [psi]', 'MOPD[bar/psi]', 'Outlet connection type', 

In [33]:
comparison = target_row == target_row2
matching_columns = comparison[comparison].index.tolist()
print("Matching columns:", matching_columns)

Matching columns: ['<Object Type>', 'Product name', 'Actuator size', 'Actuator system [mm]', 'Coil', 'Coil connection', 'Direction', 'Electrical connection', 'Enclosure rating IP', 'Frequency [Hz]', 'Function', 'Inlet connection type', 'Inlet size [in]', 'Inlet size [mm]', 'Manual operation option', 'Max. Working Pressure [bar]', 'Max. Working Pressure [psig]', 'MaxOPD[bar] 10W AC', 'MaxOPD[psi] 10W AC', 'Medium temperature range [°C] [max]', 'Medium temperature range [°C] [min]', 'MinODP [bar]', 'Outlet connection type', 'Outlet size [in]', 'Outlet size [mm]', 'Parts Program name', 'Refrigerants', 'Supply voltage 50Hz Max [V]', 'Supply voltage 50Hz Min [V]', 'Supply voltage 60Hz Max [V]', 'Supply voltage 60Hz Min [V]', 'Supply voltage [V] AC', 'Supply voltage [V] AC [max]', 'Supply voltage [V] AC [min]', 'Supply voltage [V] DC']


In [66]:

print(target_row)
print(target_row2)

<ID>                                          10125003602
<Object Type>                              Solenoid Valve
Product name                                   MDF-A03-3H
Headline                       SOLENOID VALVE MDF SERIES 
Actuator size                                          55
                                          ...            
Supply voltage 60Hz Min [V]                           NaN
Supply voltage [V] AC                                 NaN
Supply voltage [V] AC [max]                           NaN
Supply voltage [V] AC [min]                           NaN
Supply voltage [V] DC                                 NaN
Name: 0, Length: 63, dtype: object
<ID>                                          032L2073
<Object Type>                                  Product
Product name                            Solenoid valve
Headline                       Solenoid valve, EVO 105
Actuator size                                    BLANK
                                        ...         

In [70]:

# Build match matrix (True/False per column)
df[must_have_data] = df[must_have_data].fillna('BLANK').replace('', 'BLANK').replace(' ', 'BLANK')
target_row = target_row.fillna('BLANK').replace('', 'BLANK').replace(' ', 'BLANK')

exact_match = (df[must_have_data] == target_row[must_have_data])
blank_match = (df[must_have_data] == "BLANK") & (target_row[must_have_data] != "BLANK")

column_scores = exact_match.astype(float) + (blank_match.astype(float) * 0.5)



# ✅ Calculate match percentage for each row
match_counts = matches.sum(axis=1)
match_percentages = (match_counts / len(must_have_data)) * 100

row_scores = column_scores.sum(axis=1)
match_percentages = (row_scores / len(must_have_data)) * 100

top_matches = match_percentages.sort_values(ascending=False)
print(top_matches.head(50))
print(len(top_matches))

322    77.083333
299    77.083333
296    77.083333
58     77.083333
57     77.083333
54     77.083333
73     77.083333
71     77.083333
86     77.083333
83     77.083333
98     77.083333
96     77.083333
200    77.083333
197    77.083333
173    77.083333
188    77.083333
217    77.083333
90     72.916667
55     72.916667
67     72.916667
196    72.916667
68     72.916667
288    68.750000
297    68.750000
56     68.750000
59     68.750000
287    68.750000
286    68.750000
275    68.750000
277    68.750000
279    68.750000
280    68.750000
281    68.750000
282    68.750000
283    68.750000
274    68.750000
72     68.750000
273    68.750000
272    68.750000
74     68.750000
85     68.750000
81     68.750000
270    68.750000
269    68.750000
268    68.750000
267    68.750000
265    68.750000
260    68.750000
259    68.750000
257    68.750000
dtype: float64
295


In [50]:


matching_rows = df.copy()

# Print each row’s ID with its match %
for id_val, perc in zip(matching_rows['<ID>'], match_percentages):
    print(f"ID: {id_val} → {perc:.2f}% match")

ID: 032F1004 → 79.17% match
ID: 032F8132 → 75.00% match
ID: 032L1004 → 83.33% match
ID: 032L3085 → 83.33% match
ID: 032F1177 → 79.17% match
ID: 032F1178 → 79.17% match
ID: 032F1176 → 79.17% match
ID: 032F1244 → 79.17% match
ID: 032F1245 → 70.83% match
ID: 032F1254 → 79.17% match
ID: 032F1253 → 70.83% match
ID: 032F1274 → 79.17% match
ID: 032F1240 → 79.17% match
ID: 032F1273 → 70.83% match
ID: 032F1260 → 75.00% match
ID: 032F1243 → 70.83% match
ID: 032F1264 → 79.17% match
ID: 032F1269 → 75.00% match
ID: 032F1279 → 66.67% match
ID: 032F224331 → 50.00% match
ID: 032F3131 → 62.50% match
ID: 032G1020 → 79.17% match
ID: 032G1364 → 66.67% match
ID: 032L1254 → 83.33% match
ID: 032L1269 → 79.17% match
ID: 032L1245 → 75.00% match
ID: 032L1240 → 83.33% match
ID: 032L1244 → 83.33% match
ID: 032L1260 → 79.17% match
ID: 032L1253 → 70.83% match
ID: 032L1243 → 70.83% match
ID: 032L1364 → 66.67% match
ID: 032L1279 → 70.83% match
ID: 032L2243 → 54.17% match
ID: 032L2087 → 50.00% match
ID: 032L2061 → 50.

In [36]:
duplicates = df[df.duplicated(subset=must_have_data, keep=False)]
duplicates = duplicates.sort_values(by=must_have_data)

print(duplicates.iloc[2])


<ID>                                                                    032F7100
<Object Type>                                                            Product
Product name                                                      Solenoid valve
Headline                       Solenoid valve, EVR 2, Solder, ODF, 1/4 in, 0....
Actuator size                                                              BLANK
                                                     ...                        
Supply voltage 60Hz Min [V]                                                BLANK
Supply voltage [V] AC                                                      BLANK
Supply voltage [V] AC [max]                                                BLANK
Supply voltage [V] AC [min]                                                BLANK
Supply voltage [V] DC                                                      BLANK
Name: 292, Length: 63, dtype: object


In [37]:
def numeric_similarity(a, b):
    if pd.isna(a) or pd.isna(b):
        return 0
    return 1 - abs(a - b) / max(abs(a), abs(b), 1e-9)

def categorical_similarity(a, b):
    return 1.0 if str(a).lower().strip() == str(b).lower().strip() else 0.0

In [38]:
def compute_similarity(item_a, item_b):
    total_weight = sum(FEATURE_WEIGHTS.values())
    score = 0

    for feature, weight in FEATURE_WEIGHTS.items():
        val_a = item_a.get(feature)
        val_b = item_b.get(feature)

        if isinstance(val_a, (int, float)) and isinstance(val_b, (int, float)):
            sim = numeric_similarity(val_a, val_b)
        else:
            sim = categorical_similarity(val_a, val_b)

        score += sim * weight

    return score / total_weight

In [39]:
selected_a = df_a.iloc[0].to_dict()
df_b["similarity"] = df_b.apply(lambda row: compute_similarity(selected_a, row.to_dict()), axis=1)
df_b_sorted = df_b.sort_values("similarity", ascending=False)

print("Top matches:\n", df_b_sorted[["id", "similarity"]])
best_match = df_b_sorted.iloc[0]
print("\n✅ Best match found:", best_match.to_dict())

NameError: name 'df_a' is not defined